In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/profiles.csv")

## Remove unneeded variables

First, let's drop all the columns we won't use.

In [3]:
df.drop(['last_online'], axis=1, inplace=True)
df.drop([f"essay{n}" for n in np.arange(0,10)], axis=1, inplace=True)

## Refactor "diet" variable

The "diet" variable has 6 possible cuisines (anything, vegetarian, vegan, kosher, halal, and other) and 2 changers (mostly/strictly), which might create unnecessary dummy variables that won't add value to the model. So let's boil it down to diet type without the nuance.

In [4]:
df['diet'].replace(["mostly ", "strictly "], "", regex=True, inplace=True)
df['diet']

0          anything
1             other
2          anything
3        vegetarian
4               NaN
            ...    
59941           NaN
59942      anything
59943      anything
59944      anything
59945           NaN
Name: diet, Length: 59946, dtype: object

## Parse spoken languages

The same goes for the `language` variable. There are 3 modifiers for a multitude of languages. The permutation between all possible values will generate thousands of unnecessary columns. Let's just keep the languages.

In [6]:
df['speaks'].replace([" \(fluently\)", " \(okay\)", " \(poorly\)"], "", regex=True, inplace=True)
df["speaks"]

0                                            english
1                           english, spanish, french
2                               english, french, c++
3                                    english, german
4                                            english
                            ...                     
59941                                        english
59942                                        english
59943                                        english
59944    english, spanish, chinese, korean, japanese
59945                                        english
Name: speaks, Length: 59946, dtype: object

But filtering out the language modifiers isn't enough. There's only one column for all the user's spoken languages, whose combination might generate a dozen thousands dummy variables. Therefore, each `speaks` value should be parsed and gain a dummy variable of its own.

In [7]:
def parse_to_dummies(df, column):
    """
    Transforms a column with comma-separated values into new 0-or-1 columns.
    """
    expanded_values = df[column].str.split(pat=", ", expand=True)
    # see https://stackoverflow.com/questions/26977076/pandas-unique-values-multiple-columns 
    unique_labels = pd.unique(expanded_values.values.ravel('K'))
    table_shape = (df.shape[0], len(unique_labels))
    dummy_table = pd.DataFrame(np.zeros(table_shape, dtype="int"),
                              columns=[f"{column}_{label}" for label in unique_labels])
    for index,row in expanded_values.iterrows():
        for item in row:
            dummy_table.at[index, f"{column}_{item}"] = 1
   
    dummy_table.drop([f"{column}_None"], axis=1, inplace=True)
    df_drop_column = df.drop(column, axis=1)
    return pd.concat([df_drop_column, dummy_table], axis=1)

df = parse_to_dummies(df, column="speaks")

## Convert "drink" values into a numerical scale

In [ ]:
#[print(x) for x in df.columns]
df['drinks'].replace({"not at all": 0,
                     "rarely": 1,
                     "socially": 2,
                     "often": 3,
                     "very often": 4,
                     "desperately": 5}, inline=True)
df['drinks']

## Create dummy variables from categorical columns

In [ ]:
def create_dummy(df):
    """
    Creates dummy variables for all the categorial variables in a DataFrame and concatenates it
    with the original numerical columns.
    Input: pandas DataFrame
    Output: pandas DataFrame
    """
    cat_cols = df.select_dtypes(include="object").columns
    num_cols = df.select_dtypes(exclude="object").columns
    dummy_df = pd.get_dummies(df[cat_cols],
                              prefix=cat_cols,
                              prefix_sep="_",
                              dummy_na=True,
                              columns=cat_cols)
                              
    df_new = pd.concat([df[num_cols], dummy_df], axis=1)
    return df_new

In [ ]:
df = create_dummy(df)
df.head()

In [ ]:
[print(x) for x in df.columns]